### import 및 파일읽기

In [2]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
import warnings
warnings.simplefilter("ignore")

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format


##################################
company = "솔림"      # 솔림 or 대성
basedate = "사전검사" 
새채무자키사용 = True
##################################
wd = join(r"D:\3.자산\전산 dataset", company, basedate)

comp_closing = {
    # 채권 자체의 소멸
    "종료" : r"종결(?!요청)|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료",
    # 비정상-종결요청
    "종결요청" : r"종결요청|매각예정|환매예정",
    # 시효
    "시효" : r"시효|완성|만료", # 부활 제외함 231222
    # 채무자별로 체크
    "면책" : r"면책|면탈",  
    "불가" : r"추심제외|상각|포기|파산(?!\(기각\))|사망(?!\(상속승인\))", # 상각, 제외, 포기는 이미 보증인까지 고려한 결과일 수 있음 # 담당자 누락제외는 채무상태에 적용하면 안 됨.(계좌의 메모를 봐야함)
    "공채권" : r"공채권"
    }

원데이터파일명 = {"개회차주" : "개인회생_원데이터_차주", 
                "개회보증인" : "개인회생_원데이터_보증인", 
                "신복차주" : "신용회복_원데이터_차주", 
                "신복보증인" : "신용회복_원데이터_보증인", 
                "전체" : "전체_원데이터"}

전산_files = {"채무자" : "채무자조회새창", 
            "계좌" : "계좌조회새창", 
            "보증인" : "보증인새창.+새보증인상태", 
            "개인회생" : "개인회생새창", 
            "신용회복" : "신용회복_기준데이터", 
            "법조치" : "법조치조회새창", 
            "입금" : "입금조회새창"}

debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}
credit_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
# credit_dtype = {'채무자키':str, '계좌키':str, '보증인키':str, '심의차수':str, '변제금\n수취계좌':str}
# credit_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
# }
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}

def 파일명(폴더:str, 키워드:str) :
    file_list = os.listdir(폴더)
    account_filename = [file for file in file_list if re.search(키워드, file)]
    if len(account_filename) == 1 :
        return join(폴더,account_filename[0])
    else :
        print("해당 키워드가 들어간 파일이 하나가 아닙니다.")

In [46]:
# 전산 원데이터 읽기
debt = pd.read_pickle(파일명(wd, 전산_files["채무자"]))
account = pd.read_pickle(파일명(wd, 전산_files["계좌"]))
grt = pd.read_pickle(파일명(wd, 전산_files["보증인"]))
rehabilitation = pd.read_pickle(파일명(wd, 전산_files["개인회생"]))
credit = pd.read_pickle(파일명(wd, 전산_files["신용회복"]))
credit_excluded = pd.read_excel(파일명(os.path.dirname(wd), 전산_files["신용회복"]), sheet_name="일부계좌제외", dtype=credit_dtype).fillna("") # 읽는데 시간 얼마안걸리니 별도파일만들지말고 엑셀읽자
event = pd.read_pickle(파일명(wd, 전산_files["법조치"]))
deposit = pd.read_pickle(파일명(wd, 전산_files["입금"]))

# 자산정리 원데이터 파일 읽기
# 1.개인회생
re_d = pd.read_pickle(파일명(wd, 원데이터파일명["개회차주"]))
re_grt = pd.read_pickle(파일명(wd, 원데이터파일명["개회보증인"]))

# 2.신용회복
credit_d = pd.read_pickle(파일명(wd, 원데이터파일명["신복차주"]))
credit_grt = pd.read_pickle(파일명(wd, 원데이터파일명["신복보증인"]))

# 3.전체(무담보)
raw_data = pd.read_pickle(파일명(wd, 원데이터파일명["전체"]))

In [48]:
# 새채무자키 입히기
새채무자키merge용 = raw_data[["계좌키","새채무자키", "새채무상태"]]

# 보증인
grt = grt.merge(새채무자키merge용, on="계좌키")

### 데이터클렌징
- 중복 채무상태
- 중복 담당자
- 전산과 채무상태 및 담당자 불일치
- 현재원금 0원

- 개인회생 : 정상 건 중 변제오류(신고계좌, 채권번호), 최신화 실패, 회차 확인, 항고사건 결과 미확인건, 분납메모 정규화되지 않은 건
- 신용회복 : 월말보고서, 실효요청, 
- 파산 : 

#### 기초 작업 : 필요항목 정리

In [45]:
grt.새보증인상태.value_counts()

새보증인상태
정상           7488
불가            943
면책            840
종료            771
신용회복(개인)      105
개인회생(확정)       97
시효             64
개인회생(진행중)       5
신용회복(진행중)       1
Name: count, dtype: int64

In [50]:
# 기초 작업

#############################################
그룹기준 = '새채무자키' # 새채무자키 or 주민등록번호
#############################################

# 차주 ----------------------------------
# raw_data에서 필요 칼럼 불러오기
ck_debt_state = raw_data[["주민등록번호","새채무자키","채무자키","계좌키","새채무상태","채무상태","담당자","메모","반영OPB","성명","개인법인", \
    "계좌번호","원채권사","매각사구분","은행대출원금","최초원금","최초법비용","최초미수이자","현재원금","현재법비용","현재미수이자","현재연체이자","매입일",]].copy()
if 새채무자키사용 :
    ck_debt_state.insert(10, "새보증인상태by새채무자키", raw_data["새보증인상태by새채무자키"])
else : 
    ck_debt_state.insert(10, "새보증인상태", raw_data["새보증인상태"])
    
# 확실한 종결건 제거
ck_debt_state = ck_debt_state.query('새채무상태 != "종료"')

# 보증인 ----------------------------------
# 계좌 종결건 제거
ck_grt_state = grt[["주민번호인", "새채무자키","채무자키","계좌키","새보증인상태","보증인상태","담당자","종결일","종결사유","보증금액","보증한도액", "새채무상태"]].copy()
ck_grt_stat = grt.query('새채무상태!="종료"')

#### 채무상태, 담당자

##### [check]다중채무상태

In [ ]:
# 차주 -----------

# 신용회복 확정건 단일화 및 누락제외계좌 확인 건은 신복으로 처리
cond = (ck_debt_state.새채무상태 == "신용회복(개인)") | (ck_debt_state.새채무상태 == "신용회복(프리)") | (ck_debt_state.메모.str.contains(r"신용회복 (제외|누락)"))
ck_debt_state["새채무상태"] = np.where(cond, "신용회복(확정)", ck_debt_state.새채무상태)

# 메모 공채권인것도 신용회복으로 바꿀까? 현재 종결건 제외하면 하나밖에 없음.. 

# 결과
더블채무상태 = ck_debt_state.groupby(그룹기준).filter(lambda x: x['새채무상태'].nunique() > 1) # rst의 타입은 df임

# 저장
if len(더블채무상태) > 0 : 
    print(len(더블채무상태))
    더블채무상태.set_index([그룹기준,"채무자키","계좌키"], drop=True, inplace=True)
    더블채무상태.sort_index().to_excel(join(wd, basedate, "[check]더블채무상태_"+ 그룹기준 +".xlsx")) # inplace=True 하면 반환값이 없어서 오류남

21


In [ ]:
# 보증인 -----------
cond = (ck_debt_state.새채무상태 == "신용회복(개인)") | (ck_debt_state.새채무상태 == "신용회복(프리)") | (ck_debt_state.메모.str.contains(r"신용회복 (제외|누락)"))
ck_debt_state["새채무상태"] = np.where(cond, "신용회복(확정)", ck_debt_state.새채무상태)


In [ ]:
# [check]개회신복 중 매각 아닌 종결건이 있는지 확인하기 # 신복은 기준데이터에 종결건이 있을 수 있음, 풀별작업시에는 빼니까 자산정리에는 문제없음
종결된계좌키 = raw_data[raw_data.채권상태 == "종료"].계좌키
# 종결건 확인
개회_종결 = rehabilitation[rehabilitation.계좌키.isin(종결된계좌키)][["채무자키", "계좌키"]].copy()
신복_종결 = credit[credit.계좌키.isin(종결된계좌키)][["채무자키", "계좌키"]].copy()
개회_종결["구분"] = "개회"
신복_종결["구분"] = "신복"

# 종결건에 기본정보 추가
조정_종결 = pd.merge(개회_종결, 신복_종결, on=["채무자키", "계좌키", "구분"], how='outer')
조정_종결 = pd.merge(조정_종결, raw_data[["계좌키","채권상태","새채무상태","채무상태","담당자"]], on='계좌키', how='left')

# 매각 건 제외
print(f'매각건 제외 전 : {len(조정_종결)}')
매각idx = 조정_종결[(조정_종결.채무상태.str.contains('매각|DS')) | (조정_종결.담당자.str.contains('매각|DS'))].index
조정_종결 = 조정_종결.drop(매각idx)
print(f'매각건 제외 후 : {len(조정_종결)}')
display(조정_종결)
# [check]종결건 저장
if len(조정_종결)>0 : 
    조정_종결.to_excel(join(wd,basedate, "[check]매각외종결건vs조정.xlsx"), index=False)

매각건 제외 전 : 56
매각건 제외 후 : 0


,채무자키,계좌키,구분,채권상태,새채무상태,채무상태,담당자


In [ ]:
####전산_check_전체계좌.ipynb 에도 있음
account = pd.read_pickle(r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847.pkl")
new_debtKey = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\새채무자키.pkl")[["새채무자키","채무자키","계좌키"]]
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

# 필요열만 새로운 df로 & 정렬
new_debtKey = account[["새채무자키", '채무자키','계좌키','채무상태','담당자']].copy()
new_debtKey.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

###### multi_index : 조작하기는 어렵고, 엑셀로 보낼때 자동으로 병합이 되어서 보기는 좋음
x = account.set_index(['새채무자키','채무자키','계좌키'],drop=True)["채무상태"]
# 다중채무, 다중담당 T/F값 입력
x = new_debtKey.groupby('새채무자키')
new_debtKey['다중채무상태여부'] = x['채무상태'].transform(lambda s: np.any(s != s.iloc[0]))
new_debtKey['다중담당여부'] = x['담당자'].transform(lambda s: np.any(s != s.iloc[0]))
# 다중인 것들만 
check = new_debtKey[new_debtKey["다중채무상태여부"] | new_debtKey['다중담당여부']].copy()
# index설정
gc = check.set_index(['새채무자키', '채무자키'], drop=True)
#저장
gc.to_excel(r"c://Users/SL/Desktop/check_채무상태(병합).xlsx")

,채무자키,계좌키,분납키,분납자관계,분납자성명인,인가미인가,현재결과,분납중단일,중단사유,채무상태,담당자,주민번호인
0,20465138,200985188,20150117,채무자,김영준,미인가,금지,,,개인회생(진행중),개인회생,890215-1652415
1,20502439,201030117,20150053,채무자,박미리,미인가,금지,,,개인회생(진행중),개인회생,980203-2151816
2,20424736,200943368,20150077,채무자,조성윤,미인가,금지,,,개인회생(진행중),개인회생,660727-1241120
3,20514912,201045765,20150056,채무자,김홍윤,미인가,금지,,,개인회생(진행중),개인회생,800214-1473918
4,20461754,200981804,20150054,채무자,곽부창,미인가,금지,,,개인회생(진행중),개인회생,720204-1925218
...,...,...,...,...,...,...,...,...,...,...,...,...
5162,20515020,201045873,20131042,채무자,송용국,인가,인가,,,개인회생(확정),개인회생,790530-1163021
5163,20500389,201028067,20127653,채무자,한동진,인가,인가,,,개인회생(확정),개인회생,890818-1081720
5166,20514973,201045826,20130737,채무자,고규태,인가,항고,,,개인회생(확정),개인회생,840523-1540514
5169,20420689,200937575,20125267,채무자,서현무,인가,인가,,,개인회생(확정),개인회생,620318-1384717
